In [1]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
from pathlib import Path
import os
import torch
import numpy as np

/fast_storage/qnoens/conda/robilglue/lib/python3.10/site-packages/datasets/__init__.py


Load the dataset that you want to clean up by removing all the zero actions

In [2]:
root_dataset_dir = '/fast_storage/qnoens/OpenVLA/data/lerobot_task1'
dataset_name = 'test_dataset'

lerobot_dataset = LeRobotDataset(repo_id=dataset_name, root=root_dataset_dir)
meta = lerobot_dataset.meta
print(meta)

Resolving data files:   0%|          | 0/112 [00:00<?, ?it/s]

LeRobotDatasetMetadata({
    Repository ID: 'test_dataset',
    Total episodes: '112',
    Total frames: '17207',
    Features: '['next.reward', 'next.success', 'seed', 'timestamp', 'wrist_image_original', 'scene_image_original', 'wrist_image', 'scene_image', 'state', 'robot_pose', 'gripper_state', 'joints', 'action', 'frame_index', 'episode_index', 'index', 'task_index']',
})',



Decide on the path where you want the new dataset to end up

In [3]:
root_dir = Path('/fast_storage/qnoens/OpenVLA/data/full_lerobot_task1_zero_action')
use_videos = Path(root_dataset_dir + '/' + 'videos').exists()

#### Create the new dataset. 

**Note:** If the path you gave already existed, then it will deleted!

In [4]:
if root_dir.exists():
    print(f'Root directory already exists. Deleting it.')
    os.system(f'rm -rf {root_dir}')

new_dataset = LeRobotDataset.create(
    repo_id=dataset_name,
    fps=lerobot_dataset.fps,
    root=root_dir,
    features=lerobot_dataset.features,
    use_videos=use_videos,
    image_writer_processes=0,
    image_writer_threads=16,
)

Root directory already exists. Deleting it.


The `episodes_to_delete` list enables us to exclude any unwanted episodes from the original dataset and therefore won't be added to the new one.

In [5]:
episodes_to_delete = [31, 32, 33, 34, 35, 36, 37, 38, 65, 74, 91]

We will now go episode by episode over every frame within the episode to create the new episodes excluding actions that are too small and will therefore damage the performance of our model.

In [6]:
# We now go episode by episode over frame by frame
episode_count = 0
for ep_idx in range(lerobot_dataset.num_episodes):
    if ep_idx in episodes_to_delete:
        print(f"Skipping episode {ep_idx} because it is deleted.")
        continue
    episode_meta = meta.episodes[ep_idx]    
    episode_length = episode_meta['length']
    instruction = episode_meta['tasks'][0]
    lengths = [meta.episodes[i]['length'] for i in range(ep_idx)]
    start_idx = sum(lengths)

    # Iterate over frames and set action to zero
    remover_count = 0
    for frame_idx in range(start_idx, start_idx + episode_length):
        orig_frame = lerobot_dataset[frame_idx]

        # Check if action is zero
        action = orig_frame['action'].numpy()
        if np.all(np.isclose(action, 0.0, atol=2e-3)):
            remover_count += 1
            continue
        
        # Create new frame
        frame = {
            'next.reward': orig_frame['next.reward'].unsqueeze(0),
            'next.success': orig_frame['next.success'].unsqueeze(0),
            'seed': orig_frame['seed'].unsqueeze(0),  
            'wrist_image_original': orig_frame['wrist_image_original'].permute(1, 2, 0),
            'scene_image_original': orig_frame['scene_image_original'].permute(1, 2, 0),
            'wrist_image': orig_frame['wrist_image'].permute(1, 2, 0),
            'scene_image': orig_frame['scene_image'].permute(1, 2, 0),
            'state': orig_frame['state'], 
            'robot_pose': orig_frame['robot_pose'], 
            'gripper_state': orig_frame['gripper_state'].unsqueeze(0), 
            'joints': orig_frame['joints'], 
            'action': orig_frame['action'],
            'task': instruction
        }

        # Add frame to new dataset
        new_dataset.add_frame(frame)


    # Add episode to new dataset
    episode_count += 1
    print(f'Removed {remover_count} frames from episode {episode_count}/{lerobot_dataset.num_episodes - len(episodes_to_delete)}')
    new_dataset.save_episode()




Removed 74 frames from episode 1/101


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Removed 22 frames from episode 2/101


Map:   0%|          | 0/146 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Removed 18 frames from episode 101/101


Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]